In [1]:
import sys
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as lg
import matplotlib
import matplotlib.pyplot as plt
from cfd_common import *
from math import *

In [2]:
%matplotlib qt

In [3]:
plt.ion()

In [7]:

##### Domain #####

# Domain size
Lx = 2.
Ly = 2.

# Physical grid size
Nx_phys = 50
Ny_phys = 50
Δx = Lx/Nx_phys
Δy = Ly/Ny_phys
# Numerical grid size, including ghost points
Nx = Nx_phys + 2
Ny = Ny_phys + 2

# Physical grid (x,y)
# Due to the "staggered-like" approach to boundary conditions with ghost points,
#  the border go through the middle of border cells
x = np.linspace(Δx/2, Lx-Δx/2, Nx_phys) 
y = np.linspace(Δy/2, Ly-Δy/2, Ny_phys)
[xx,yy] = np.meshgrid(x,y) 

##### Fields #####

# Velocity field (u,v)[i,j], initialized to "zero"
u = np.zeros((Nx,Ny))
v = np.zeros((Nx,Ny))
vrand_ampli = 0.1
v += vrand_ampli * ( 1-2*np.random.rand(Nx,Ny) ) # random vertical velocity to speed up the instability

# Pressure field press[i,j] and its gradient (dx_press,dy_press)
press = np.zeros((Nx,Ny))
dx_press = np.zeros((Nx,Ny))
dy_press = np.zeros((Nx,Ny))

# Density field
ρ = np.ones((Nx,Ny))
ρ[:, 4*Ny//10 : 8*Ny//10] = 5

def VelocityGhostPoints(u,v):
    # ya pas cette histoire de "frontière = au milieu des cellules de bord ?"
    ### left
    u[0, :] = 0
    v[0, :] = 0
    ### right    
    u[-1, :] = u[-2, :]    # outflow condition => dérivée nulle
    v[-1, :] = v[-2, :]    # outflow condition => dérivée nulle
    v[-1, :] = 0  # no slip
    u[-1, :] = 0  # imperméabilité
    ### bottom
    u[:,  0] = 0  # no slip
    v[:,  0] = 0  # imperméabilité
    ### top     
    u[:, -1] = 0#u[:, -2]
    v[:, -1] = 0#v[:, -2]

  
    
###### CONSTRUCTION des matrices et LU decomposition

### Matrix construction for projection step
LAPN = FD_2D_Laplacian_matrix (Nx_phys, Ny_phys, Δx, Δy, BCdir_left=False, BCdir_right=True, BCdir_top=False, BCdir_bot=False) 
LUPN = LUdecomposition(LAPN)


### ATTENTION: dt_init calculer la CFL a chaque iteration... 
Δt = 0.00001

t = 0. # total time

# parameters (Reynolds number)
Re = 10


fig = plt.figure(1, figsize=(14,7))

loop_continue = True
def on_close (event):
    global loop_continue
    loop_continue = False
fig.canvas.mpl_connect('close_event', on_close)


niter = 0
while loop_continue:

    ###### Advection semi-Lagrangienne
    adv_u = SemiLag(u,v, u, Δx,Δy,Δt)
    adv_v = SemiLag(u,v, v, Δx,Δy,Δt)
    
    ρ = SemiLag(u,v, ρ, Δx,Δy,Δt)
    # c'est totalement instable avec SemiLag2, pourquoi ????
    
    # ghost points for density : Neumann ∂ρ=0 boundary condition
    ρ[ 0, :] = ρ[ 1, :] # left
    ρ[-1, :] = ρ[-2, :] # right
    ρ[:,  0] = ρ[:,  1] # bottom
    ρ[:, -1] = ρ[:, -2] # top

    ###### Diffusion step
    ustar = adv_u + Δt*Laplacian(u, Δx,Δy)/Re/ρ
    vstar = adv_v + Δt*Laplacian(v, Δx,Δy)/Re/ρ
    
    g = 1
    α = 0*pi/4
    ustar += +sin(α) * g * Δt
    vstar += -cos(α) * g * Δt
    
    ###### Ghost points update
    VelocityGhostPoints(ustar,vstar)
 
    ### Update divstar 
    divstar = Divergence(ustar,vstar, Δx,Δy)
    divstar = divstar - np.mean(divstar[1:-1,1:-1])
    
    ### Solving the linear system
    press[1:-1,1:-1] = Resolve(LUPN, RHS=divstar[1:-1,1:-1])

    ### update Pressure ghost points 
    press[0,  :] = press[1,  :] # left
    press[-1, :] = -press[-2, :] # right 
    press[:,  0] = press[:,  1] # bottom 
    press[:, -1] = press[:, -2] # top 

    ### Update gradphi
    dx_press[1:-1, :] = (press[2:, :] - press[:-2, :])/Δx/2
    dy_press[:, 1:-1] = (press[:, 2:] - press[:, :-2])/Δy/2

    ### Project u
    u = ustar - dx_press / ρ
    v = vstar - dy_press / ρ

    VelocityGhostPoints(u,v)
    
    
    

    if (niter%1000 == 0):
        
        fig.clear()
        (ax1,ax2) = fig.subplots(nrows=1, ncols=2, sharey=True)
        
        def plot_vel (ax, modx=1, mody=1, vmax=10):
            ax.quiver(xx[::modx,::mody], yy[::modx,::mody], np.transpose(u[1:-1:modx,1:-1:mody]), np.transpose(v[1:-1:modx,1:-1:mody]), scale=vmax*2)
            ax.set_aspect('equal', adjustable='box')
    
        fig.suptitle(r"Velocity, Density and Pression fields at $t={:.4f}$".format(t))
        
        ax1.clear()
        ax1.imshow(np.transpose(ρ[1:-1,1:-1]), origin='lower', extent=(0,Lx,0,Ly), cmap=plt.cm.RdPu, vmin=0, vmax=6)
        plot_vel(ax1)
        
        ax2.clear()
        im = ax2.imshow(np.transpose(press[1:-1,1:-1]), origin='lower', extent=(0,Lx,0,Ly), cmap='plasma')
        fig.colorbar(im)
        plot_vel(ax2)
        
#        my_cmap = plt.cm.gnuplot2_r(np.arange(plt.cm.gnuplot2_r.N))
#        my_cmap = 0.5 * (1+my_cmap)
#        my_cmap = matplotlib.colors.ListedColormap(my_cmap)
#        ax1.imshow(np.transpose(np.sqrt(u[1:-1,1:-1]**2 + v[1:-1,1:-1]**2)), origin='lower', extent=(0,Lx,0,Ly), cmap=my_cmap, vmin=0, vmax=2)
#        plot_vel(ax1)
    

        fig.tight_layout()
#       plt.savefig('blah/{:04d}.png'.format((niter+1)//modulo))
        plt.draw()
        
        mpl_pause_background(0.001)
        
    t += Δt
    niter += 1